In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
data_path = '../russell_protein/protein_mpnn/tests/fixtures/pdb_2021aug02_sample'

import sys
sys.path.append('../')

from russell_protein.protein_mpnn.dataset import (
    AnishchankoMultiChainDataset,
    build_protein_mpnn_input_structure_dict
)

from russell_protein.protein_mpnn.visualize import (
    data_to_pdb
)


ds = AnishchankoMultiChainDataset(
    'valid',
    data_path,
)

avail_data = []
for x in range(len(ds)):
    try:
        data = ds[x]
        avail_data.append(data)
    except Exception as e:
        continue

avail_data[0].keys()
len(avail_data[0]['seq_chain_A']), avail_data[0]['coords_chain_A']['N_chain_A'].shape

tensor([0, 1], dtype=torch.int32)
tensor([0, 1], dtype=torch.int32)
tensor([0, 1, 2, 3], dtype=torch.int32)
tensor([0, 1, 2, 3], dtype=torch.int32)
tensor([0, 1, 2, 3], dtype=torch.int32)
tensor([0, 1, 2, 3], dtype=torch.int32)
tensor([0, 1], dtype=torch.int32)
tensor([0, 1], dtype=torch.int32)
tensor([0, 1, 2, 3, 4, 5], dtype=torch.int32)
tensor([0, 1, 2, 3, 4, 5], dtype=torch.int32)
tensor([0, 1, 2, 3, 4, 5], dtype=torch.int32)
tensor([0, 1, 2, 3, 4, 5], dtype=torch.int32)
tensor([0, 1, 2, 3, 4, 5], dtype=torch.int32)
tensor([0, 1, 2, 3, 4, 5], dtype=torch.int32)


/Users/rmcl/Documents/PhD/development/protein-mpnn-explore/notebooks/../russell_protein/protein_mpnn/dataset.py:138: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  meta = tor

(550, torch.Size([550, 3]))

In [ ]:
print(avail_data[0]['coords_chain_A'].keys())

avail_data[0]['coords_chain_A']['N_chain_A'].shape

dict_keys(['N_chain_B', 'CA_chain_B', 'C_chain_B', 'O_chain_B'])


torch.Size([550, 3])

In [28]:
avail_data[0]['seq_chain_A'] == avail_data[0]['seq_chain_B']

True

In [10]:
import torch

data = avail_data[0]['coords_chain_A']

r = torch.stack([
    data['N_chain_A'],
    data['CA_chain_A'],
    data['C_chain_A'],
    data['O_chain_A'],
])

In [ ]:
possible_chains = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
aa_atoms = ['N', 'CA', 'C', 'O']

XDIM_N = 0
XDIM_CA = 1
XDIM_C = 2
XDIM_O = 3

data = avail_data[0]

all_chain_data = {}
for chain in possible_chains:

    key = f'coords_chain_{chain}'
    if key not in data:
        break

    for aa_atom in aa_atoms:
        aa_atom_key = f'{aa_atom}_chain_{chain}'
        if aa_atom_key not in data[key]:
            raise Exception(f'{aa_atom_key} not in data. Cannot proceed- data invalid')

        if aa_atom not in all_chain_data:
            all_chain_data[aa_atom] = []

        atom_coords = data[key][aa_atom_key]
        all_chain_data[aa_atom].append(atom_coords)

concat_atom_chains = {}
for aa_atom in aa_atoms:
    concat_atom_chains[aa_atom] = torch.concat(all_chain_data[aa_atom], dim=0)


# (num aa, num atoms, 3d coords)
Xi = torch.stack([
    concat_atom_chains['N'],
    concat_atom_chains['CA'],
    concat_atom_chains['C'],
    concat_atom_chains['O'],
], dim=1)


# Replace NaNs with 0
# TODO: I THINK THIS IS WHAT WE WANT HERE. SEEMS LIKE PAPER DOES IT LATER IN MODEL
Xi[torch.isnan(Xi)] = 0

# add one more dimension
X = Xi.unsqueeze(0)

X.shape

torch.Size([1, 1100, 4, 3])

In [ ]:
def calculate_Cb(X):
    """Calculate the position of virtual atom Cb for a batch of protein structures.

    Input:
    X: (batch, num aa, num atoms, 3d coords)

    atoms_dim:
        0: N
        1: CA
        2: C
        3: O

    This is calculated here:
    https://github.com/dauparas/ProteinMPNN/blob/main/protein_mpnn_utils.py#L967

    """
    # Add additional atom for "Cb" virtual atom
    b = X[:, :, XDIM_CA, :] - X[:, :, XDIM_N, :]
    c = X[:, :, XDIM_C, :] - X[:, :, XDIM_CA, :]
    a = torch.cross(b, c, dim=-1)
    Cb = -0.58273431*a + 0.56802827*b - 0.54067466*c + X[:, :, XDIM_CA, :]

    return Cb

def get_nearest_neighbors(X_atoms, k):
    """Return the k nearest neighbors of each atom in X_atoms.

    For example if the Ca of each amino acid is given, this function will return the k nearest
    Ca atoms in the protein for each Ca atom.

    Inputs:
        X_atoms: (batch, num atoms, 3d coords)
        k: int: number of nearest neighbors to return
    """
    eps = 1e-6

    # Create a BxNxN pairwise mask
    mask_2D = torch.unsqueeze(mask,1) * torch.unsqueeze(mask,2)

    # Compute the pairwise differences between every pair of points in X
    dCa = torch.unsqueeze(Ca,1) - torch.unsqueeze(Ca,2)
    D = mask_2D * torch.sqrt(torch.sum(dCa**2, 3) + eps)

    D_max, _ = torch.max(D, -1, keepdim=True)
    D_adjust = D + (1. - mask_2D) * D_max
    D_neighbors, E_idx = torch.topk(
        D_adjust,
        min(top_k, X.shape[1]),
        dim=-1,
        largest=False)

    return D_neighbors, E_idx


In [56]:
#b = X[:,:,1,:] - X[:,:,0,:]
#c = X[:,:,2,:] - X[:,:,1,:]
#a = torch.cross(b, c, dim=-1)
#Cb = -0.58273431*a + 0.56802827*b - 0.54067466*c + X[:,:,1,:]

N = X[:,:,0,:]
Ca = X[:,:,1,:]
C = X[:,:,2,:]
O = X[:,:,3,:]

NameError: name 'X' is not defined

In [129]:
X = Xi.unsqueeze(0)

mask = X.sum(dim=(2, 3)).isfinite().to(torch.float32)
mask.shape

torch.Size([1, 1100])

In [ ]:
# look at the _dist function in ProteinFeature

#def _dist(self, X, mask, eps=1E-6):

# X - (batch, num aa, num atoms, 3d coords)
# mask - (batch, num aa)
# eps - 1e-6
top_k = 30
Ca = X[:,:,1,:]
eps = 1e-6

print(Ca.shape, mask.shape)

# Create a BxNxN pairwise mask
mask_2D = torch.unsqueeze(mask,1) * torch.unsqueeze(mask,2)


# Compute the pairwise differences between every pair of points in X
dCa = torch.unsqueeze(Ca,1) - torch.unsqueeze(Ca,2)

print(mask_2D.shape, dCa.shape)

D = mask_2D * torch.sqrt(torch.sum(dCa**2, 3) + eps)

print(D.shape)

D_max, _ = torch.max(D, -1, keepdim=True)
D_adjust = D + (1. - mask_2D) * D_max
D_neighbors, E_idx = torch.topk(
    D_adjust,
    min(top_k, X.shape[1]),
    dim=-1,
    largest=False)

print('dadj', D_adjust.shape)
print(D_neighbors.shape)
print(E_idx.shape)

"""

return D_neighbors, E_idx
"""

torch.Size([1, 1100, 3]) torch.Size([1, 1100])
torch.Size([1, 1100, 1100]) torch.Size([1, 1100, 1100, 3])
torch.Size([1, 1100, 1100])
dadj torch.Size([1, 1100, 1100])
torch.Size([1, 1100, 30])
torch.Size([1, 1100, 30])


'\n\nreturn D_neighbors, E_idx\n'

In [171]:
torch.eps

AttributeError: module 'torch' has no attribute 'eps'

In [ ]:
E_idx[0,1000,:]

tensor([ 50,  49,  51,  48,  52,  53,  47, 224, 312, 311, 314,  61,  62,  63,
        222, 313, 223,  46, 227, 228,  56,  54, 260,  60, 264, 261, 225, 316,
        310, 315])

In [136]:
torch.unsqueeze(X,1).shape, torch.unsqueeze(X,2).shape

(torch.Size([1, 1, 1100, 4, 3]), torch.Size([1, 1100, 1, 4, 3]))

In [ ]:
import torch.nn as nn

class ProteinFeatures(nn.Module):
    """
    We represent protein structure in terms of an attributed graph G = (V, E) with
    node features V = {v1, . . . , vN } describing each residue (amino acid, which
        are the letters which compose a protein sequence) and
    edge features E = {eij}i6=j capturing relationships between them


    """


    def __init__(
        self,
        num_edge_features,
        num_node_features,
        num_positional_embeddings=16,
        num_rbf = 16,
        top_k = 30,
        augment_eps = 0.,
        num_chain_embeddings = 16
    ):
        """"""
        super(ProteinFeatures, self).__init__()

        node_in = 6
        edge_in = num_positional_embeddings + num_rbf * 25

        self.position_embedding = PositionalEncodings(num_positional_embeddings)
        self.edge_embedding = nn.Linear(edge_in, num_edge_features, bias=False)
        self.norm_edges = nn.LayerNorm(num_edge_features)


    def forward(self, X, mask, residue_idx, chain_labels):
        """

        X: (batch, num aa, num atoms, 3d coords)
        mask: (batch, num aa)
        ???residue_idx: (batch, num aa)
        ???chain_labels: (batch, num aa)
        """
        Cb = calculate_Cb(X)

        Ca = X[:,:,XDIM_CA,:]
        N = X[:,:,XDIM_N,:]
        C = X[:,:,XDIM_C,:]
        O = X[:,:,XDIM_O,:]


In [ ]:
def get_rbf_pairs(Ca, N, C, O, Cb):
    pairs = [
        (N, N),     #N-N
        (C, C),     #C-C
        (O, O),     #O-O
        (Cb, Cb),   #Cb-Cb
        (Ca, N),    #Ca-N
        (Ca, C),    #Ca-C
        (Ca, O),    #Ca-O
        (Ca, Cb),   #Ca-Cb
        (N, C),     #N-C
        (N, O),     #N-O
        (N, Cb),    #N-Cb
        (Cb, C),    #Cb-C
        (Cb, O),    #Cb-O
        (O, C),     #O-C
        (N, Ca),    #N-Ca
        (C, Ca),    #C-Ca
        (O, Ca),    #O-Ca
        (Cb, Ca)    #Cb-Ca
        (C, N)      #C-N
        (O, N)      #O-N
        (Cb, N)     #Cb-N
        (C, Cb)     #C-Cb
        (O, Cb)     #O-Cb
        (C, O)      #C-O
    ]

    return [

        for pair in pairs
    ]

In [178]:
t = torch.tensor([[1, 2], [3, 4], [5, 6]])

torch.gather(t, 1, torch.tensor([[1, 1], [1, 0], [0, 0]]))

tensor([[2, 2],
        [4, 3],
        [5, 5]])